<a href="https://colab.research.google.com/github/JoseMelvin-Calleja/Home_Sales/blob/main/Home_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 2s (140 kB/s)
Reading package lists... Done


In [2]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)


home_sales_df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("homeSales")

In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("select YEAR(date) as year, ROUND(AVG(price),2) as avg_price from homeSales where bedrooms == 4 group by YEAR(date) order by YEAR(date)").show()


+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("select YEAR(date) as year, ROUND(AVG(price),2) as avg_price from homeSales where bedrooms == 3 and bathrooms == 3 group by YEAR(date) order by YEAR(date)").show()



+----+---------+
|year|avg_price|
+----+---------+
|2019|287287.82|
|2020|294204.16|
|2021|294211.46|
|2022|292725.69|
+----+---------+



In [14]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query = """
  SELECT 
    date_built,
    ROUND(AVG(price),2) as avg_price 
  from homeSales 
  where bedrooms == 3 and bathrooms == 3 and floors == 2 and sqft_lot >= 2000
  group by date_built order by date_built
"""
spark.sql(query).show()



+----------+---------+
|date_built|avg_price|
+----------+---------+
|      2010|280447.23|
|      2011|281413.45|
|      2012|295858.68|
|      2013|295142.13|
|      2014|294195.13|
|      2015|291788.36|
|      2016| 287812.6|
|      2017|282026.59|
+----------+---------+



In [9]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """
  SELECT 
    ROUND(AVG(price),2) as avg_price 
  from homeSales 
  where price >= 350000
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+---------+
|avg_price|
+---------+
|473796.26|
+---------+

--- 0.5774424076080322 seconds ---


In [10]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales as select * from homeSales")

DataFrame[]

In [11]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [12]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query = """
  SELECT 
    ROUND(AVG(price),2) as avg_price 
  from home_sales 
  where price >= 350000
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))


+---------+
|avg_price|
+---------+
|473796.26|
+---------+

--- 0.5093729496002197 seconds ---


The runtime is slightly faster taking the query from the cached table than the uncahced table.

In [15]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_sales_df.write.parquet('date_built', mode = 'overwrite')

In [16]:
# 11. Read the formatted parquet data.
p_df = spark.read.parquet('date_built')

In [17]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_date_built')

In [18]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

query = """
  SELECT 
    ROUND(AVG(price), 2) as avg_price
  FROM p_date_built
  where price >= 350000
"""
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+---------+
|avg_price|
+---------+
|473796.26|
+---------+

--- 1.073775291442871 seconds ---


Running the query from the parquet data, we get a longer runtime than the cached version.

In [19]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [20]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False